# Sentiment Analysis App with Huggingface Distributed GPU Training and PyTorch Lightning

In this notebook, we will reimplement the work done in the SageMaker Project notebook, but with a language model obtained from Huggingface and using PyTorch Lightning to train the model. We are doing this because the model trained in the SageMaker Notebook used an LSTM, which is quite outdated for language models.

## Setup

In [1]:
pip install -U sagemaker

     |████████████████████████████████| 436 kB 19.0 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.52.1-py2.py3-none-any.whl size=613967 sha256=b74d8e2c3ac52265c28a5ec33da63f96e46a9f066374f6e1fd7cf04038d13567
  Stored in directory: /home/ec2-user/.cache/pip/wheels/97/15/5f/6750101f487779c36b388c09b948bc985c5583cd85d3f712a6
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.45.0
    Uninstalling sagemaker-2.45.0:
      Successfully uninstalled sagemaker-2.45.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sagemaker
from sagemaker.huggingface import HuggingFace
sagemaker.__version__

'2.52.1'

In [3]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# gets role for executing training job
role = sagemaker.get_execution_role()
hyperparameters = {
	'model_name_or_path':'distilbert-base-uncased',
	'output_dir':'/opt/ml/model',
    'dataset_name': 'imdb',
    'do_train': True,
    'do_eval': True,
    'per_device_train_batch_size': 12,
    'num_train_epochs': 5,
    'max_seq_length': 128,
    'fp16': True,
    'pad_to_max_length': True,
}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'}

# configuration for running training on smdistributed Data Parallel
# smdistributed = SageMaker Distributed
distribution = {'smdistributed': {'dataparallel':{'enabled': True}}}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
	entry_point='run_glue.py',
	source_dir='./examples/pytorch/text-classification',
	instance_type='ml.p3.8xlarge', # has 4 GPUs
	instance_count=2, # changed to 2 instances
	role=role,
	git_config=git_config,
	transformers_version='4.6.1',
	pytorch_version='1.7.1',
	py_version='py36',
	hyperparameters = hyperparameters
)

# starting the train job
huggingface_estimator.fit()

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p3.8xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 2 Instances. Please contact AWS support to request an increase for this limit.

In [ ]:
!pip install transformers torch==1.6.0

In [ ]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = 'imdb_sentiment_distributed_transformer'

os.makedirs(local_path, exist_ok=True)

# download model S3
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path=local_path, # local path where *.tar.gz will be saved
)

# unzip model
tar = tarfile.open(f'{local_path}/model.tar.gz', 'r:gz')
tar.extractall(path=local_path)
tar.close()
os.remove(f'{local_path}/model.tar.gz')

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model=AutoModelForSequenceClassification.from_pretrained(local_path)
tokenizer=AutoTokenizer.from_pretrained(local_path)

clf = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
review = ''

In [ ]:
clf(review)